In [1]:
# requirements: pdfplumber, pandas, tabula-py , numpy
import re
import pdfplumber
import pandas as pd
import numpy as np
from pathlib import Path

# SQLAlchemy + pyodbc para SQL Server
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Connection
import pyodbc


In [2]:
PDFS = [
    "pdfs/PA_OTOÑO_2025_SEMESTRAL_ICC.pdf",
    "pdfs/PA_OTOÑO_2025_SEMESTRAL_ITI.pdf",
    "pdfs/PA_OTOÑO_2025_SEMESTRAL_LCC.pdf",
]

In [3]:
def clean_header(cols):
    return [re.sub(r"\s+", " ", c).strip().lower() for c in cols]

def extract_tables_pdfplumber(pdf_path):
    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            for table in page.extract_tables():
                if not table or len(table) < 2: 
                    continue
                header = clean_header(table[0])
                # heurística: columnas esperadas
                if {"nrc","clave","materia","días","hora","profesor","salón"}.issubset(set(header)) or \
                   {"nrc","clave","materia","dias","hora","profesor","salon"}.issubset(set(header)):
                    for r in table[1:]:
                        if r and any(x for x in r):
                            rows.append(dict(zip(header, r)))
    return pd.DataFrame(rows) if rows else pd.DataFrame()

def extract_all():
    frames = []
    for p in PDFS:
        if Path(p).exists():
            df = extract_tables_pdfplumber(p)
            if not df.empty:
                df["origen_pdf"] = Path(p).name
                frames.append(df)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

raw = extract_all()


In [4]:
# Normalización de encabezados frecuentes
raw = raw.rename(columns={
    "dias": "días", "salon": "salón"
})

# Limpieza de profesor
def normalizar_profesor(x: str):
    if not isinstance(x, str):
        return None
    x = re.sub(r"\s+", " ", x).strip()
    x = x.replace(" - ", " ")
    return x.title()

raw["profesor"] = raw["profesor"].apply(normalizar_profesor)

# Pasar todo a minúsculas y eliminar espacios
raw.columns = [c.strip().lower() for c in raw.columns]

# Asegurar que exista la columna 'hora'
if "horario" in raw.columns and "hora" not in raw.columns:
    raw.rename(columns={"horario": "hora"}, inplace=True)
elif "hora " in raw.columns:
    raw.rename(columns={"hora ": "hora"}, inplace=True)
elif "hora\n" in raw.columns:
    raw.rename(columns={"hora\n": "hora"}, inplace=True)
elif "h" in raw.columns:  # casos raros de extracción truncada
    raw.rename(columns={"h": "hora"}, inplace=True)

# Si sigue sin existir, crear una columna vacía para evitar errores posteriores
if "hora" not in raw.columns:
    raw["hora"] = None

# ---------------------------------------------------------
# Función para parsear horas de forma robusta
# ---------------------------------------------------------
def parse_hora(rango):
    """Parses hour ranges like '0700-0859', '07:00-08:59', '7:00 - 8:59'."""
    if not isinstance(rango, str):
        return pd.Series([None, None, None])
    
    s = rango.strip()
    s = re.sub(r"\s+", "", s)
    if s.lower() in ["nan", "none", ""]:
        return pd.Series([None, None, None])

    patron = r"(\d{1,2}):?(\d{2})-(\d{1,2}):?(\d{2})"
    m = re.match(patron, s)
    if not m:
        return pd.Series([None, None, None])

    h1, m1, h2, m2 = map(int, m.groups())
    start = pd.to_datetime(f"{h1:02d}:{m1:02d}", format="%H:%M", errors="coerce")
    end   = pd.to_datetime(f"{h2:02d}:{m2:02d}", format="%H:%M", errors="coerce")

    if pd.isna(start) or pd.isna(end):
        return pd.Series([None, None, None])
    duracion = int((end - start).total_seconds() / 60)
    if duracion <= 0:
        return pd.Series([None, None, None])

    return pd.Series([start.time(), end.time(), duracion])

# ---------------------------------------------------------
# Aplicar la función y crear las tres columnas
# ---------------------------------------------------------
raw[["hora_inicio", "hora_fin", "duracion_min"]] = raw["hora"].apply(parse_hora)


In [5]:
DIA_MAP = {"L":"Lunes","A":"Martes","M":"Miercoles",
           "J":"Jueves","V":"Viernes","S":"Sábado"}

# Expandir por múltiples días en una sola fila (si aplica)
def explotar_por_dia(df):
    out = []
    for _, row in df.iterrows():
        dias = str(row["días"]).replace(" ", "")
        if "," in dias:
            tokens = dias.split(",")
        else:
            tokens = list(dias)  # "AJL" -> ["A","J","L"]
        for d in tokens:
            r = row.copy()
            r["dia_codigo"] = d
            r["dia_semana"] = DIA_MAP.get(d, d)
            out.append(r)
    return pd.DataFrame(out)

curated = explotar_por_dia(raw)


In [6]:
# Salón -> edificio/aula: "1CCO4/203" -> edificio=1CCO4, aula=203
def split_salon(s):
    if not isinstance(s, str): return pd.Series([None, None, None])
    s = s.strip()
    m = re.match(r"([^/]+)/?(\w+)?", s)
    if not m: return pd.Series([s, None, s])
    edificio, aula = m.group(1), m.group(2)
    return pd.Series([edificio, aula, s])

curated[["edificio","aula","codigo_salon"]] = curated["salón"].apply(split_salon)

# Dimensiones (surrogate keys)
def build_dim(df, col_key, cols_keep, start_id=1, name_id="id"):
    d = df[cols_keep].drop_duplicates().reset_index(drop=True)
    d.insert(0, name_id, range(start_id, start_id+len(d)))
    return d

dim_docente = build_dim(curated, "profesor", ["profesor"], name_id="id_docente")
dim_materia = build_dim(curated, "materia", ["clave","materia"], name_id="id_materia")
dim_espacio = build_dim(curated, "codigo_salon", ["edificio","aula","codigo_salon"], name_id="id_espacio")

# dim_tiempo por fila (día + rango)
dim_tiempo = curated[["dia_codigo","dia_semana","hora_inicio","hora_fin"]].drop_duplicates().reset_index(drop=True)
dim_tiempo.insert(0, "id_tiempo", range(1, len(dim_tiempo)+1))

# Hechos (join a dimensiones)
def map_id(df, dim, key_cols_df, key_cols_dim, id_col):
    if isinstance(key_cols_df, str):
        key_cols_df = [key_cols_df]
    if isinstance(key_cols_dim, str):
        key_cols_dim = [key_cols_dim]

    df["_key_"] = df[key_cols_df].astype(str).agg("|".join, axis=1)
    dim["_key_"] = dim[key_cols_dim].astype(str).agg("|".join, axis=1)

    merged = df.merge(dim[["_key_", id_col]], on="_key_", how="left", validate="m:1")
    result = merged[id_col].values

    df.drop(columns="_key_", inplace=True, errors="ignore")
    dim.drop(columns="_key_", inplace=True, errors="ignore")

    return result

hechos = curated.copy()
hechos["id_docente"] = map_id(hechos, dim_docente, "profesor", "profesor", "id_docente")
hechos["id_materia"] = map_id(hechos, dim_materia, ["clave","materia"], ["clave","nombre_materia" if "nombre_materia" in dim_materia.columns else "materia"], "id_materia")
hechos["id_espacio"] = map_id(hechos, dim_espacio, "codigo_salon", "codigo_salon", "id_espacio")
hechos = hechos.merge(dim_tiempo, on=["dia_codigo","dia_semana","hora_inicio","hora_fin"], how="left")

print("Filas sin id_materia:", hechos["id_materia"].isna().sum())

hechos_clase = hechos[[
    "id_docente"    ,"id_materia","id_espacio","id_tiempo",
    "nrc","clave","secc" if "secc" in hechos.columns else "sección" if "sección" in hechos.columns else "días",
    "duracion_min"
]].rename(columns=lambda c: {"días":"seccion"}.get(c, c))


Filas sin id_materia: 0


In [7]:
#-------------------------------------------------------------
# Guardar los resultados intermedios en archivos CSV
#-------------------------------------------------------------
output_dir = Path("data_export")
output_dir.mkdir(exist_ok=True)

#print(">> Guardando archivos CSV intermedios en ./data_export/")

raw.to_csv(output_dir / "data_raw.csv", index=False, encoding="utf-8-sig")
curated.to_csv(output_dir / "data_curated.csv", index=False, encoding="utf-8-sig")
hechos_clase.to_csv(output_dir / "data_hechos.csv", index=False, encoding="utf-8-sig")
dim_docente.to_csv(output_dir / "dim_docente.csv", index=False, encoding="utf-8-sig")
dim_materia.to_csv(output_dir / "dim_materia.csv", index=False, encoding="utf-8-sig")
dim_espacio.to_csv(output_dir / "dim_espacio.csv", index=False, encoding="utf-8-sig")
dim_tiempo.to_csv(output_dir / "dim_tiempo.csv", index=False, encoding="utf-8-sig")

print(">> Archivos CSV guardados correctamente en la carpeta 'data_export'")


>> Archivos CSV guardados correctamente en la carpeta 'data_export'


In [8]:
# -------------------------------------------------------------
# Limpieza y validación robusta de horas antes de la carga
# -------------------------------------------------------------

# Normaliza tipos de hora
raw["hora_inicio"] = pd.to_datetime(raw["hora_inicio"], errors="coerce").dt.time
raw["hora_fin"] = pd.to_datetime(raw["hora_fin"], errors="coerce").dt.time

# Quita filas sin hora válida
raw = raw.dropna(subset=["hora_inicio", "hora_fin"]).reset_index(drop=True)

# Filtra filas donde hora_fin <= hora_inicio
def es_valida(row):
    try:
        return row["hora_fin"] > row["hora_inicio"]
    except Exception:
        return False

mask_validas = raw.apply(es_valida, axis=1)
raw = raw.loc[mask_validas].copy().reset_index(drop=True)

# Recalcula duración por consistencia
def calcular_duracion(row):
    try:
        start = pd.to_datetime(str(row["hora_inicio"]), format="%H:%M:%S")
        end = pd.to_datetime(str(row["hora_fin"]), format="%H:%M:%S")
        dur = int((end - start).total_seconds() / 60)
        return dur if dur > 0 else None
    except Exception:
        return None

raw["duracion_min"] = raw.apply(calcular_duracion, axis=1)
raw = raw.dropna(subset=["duracion_min"]).reset_index(drop=True)


In [14]:
# ---------------------------------------------------------------------------
# 🔧 CONFIGURACIÓN GLOBAL
# ---------------------------------------------------------------------------
SERVER_NAME = "ANDYPAVON" 
DATABASE_NAME = "horariosCubo" 

# Cadena de conexión
CONNECTION_STRING = (
    f"mssql+pyodbc:///?odbc_connect="
    f"Driver={{ODBC Driver 17 for SQL Server}};" 
    f"Server={SERVER_NAME};"
    f"Database={DATABASE_NAME};"
    f"Trusted_Connection=yes;" 
)

# ---------------------------------------------------------------------------
# 🔗 CONEXIÓN A SQL SERVER
# ---------------------------------------------------------------------------
try:
    engine = create_engine(CONNECTION_STRING)
    conn = engine.connect()
    conn.execute(text("SELECT 1")) 
    print("✅ Conexión a SQL Server establecida correctamente (Windows Auth).")

except Exception as err:
    print(f"❌ Error al conectar a SQL Server (Verifica el driver ODBC): {err}")
    raise SystemExit()


# ---------------------------------------------------------------------------
# 🧱 CREACIÓN DE TABLAS (DDL SIN CLAVES FORÁNEAS INICIALES)
# ---------------------------------------------------------------------------

# PASO 1: ELIMINAR TABLAS EN ORDEN INVERSO DE DEPENDENCIA
drop_statements = [
    """
    IF OBJECT_ID('hechos_clase', 'U') IS NOT NULL DROP TABLE hechos_clase;
    """,
    """
    IF OBJECT_ID('dim_docente', 'U') IS NOT NULL DROP TABLE dim_docente;
    IF OBJECT_ID('dim_materia', 'U') IS NOT NULL DROP TABLE dim_materia;
    IF OBJECT_ID('dim_espacio', 'U') IS NOT NULL DROP TABLE dim_espacio;
    IF OBJECT_ID('dim_tiempo', 'U') IS NOT NULL DROP TABLE dim_tiempo;
    """,
]

print(">> Eliminando tablas existentes...")
for ddl in drop_statements:
    conn.execute(text(ddl))
conn.commit()
print("✅ Tablas eliminadas correctamente.")


# PASO 2: CREAR TABLAS (Dimensiones antes que Hechos, sin FKs en Hechos)
ddl_statements = [
    """
    CREATE TABLE dim_docente (id_docente INT PRIMARY KEY, nombre_completo NVARCHAR(200))
    """,
    """
    CREATE TABLE dim_materia (id_materia INT PRIMARY KEY, clave VARCHAR(50), nombre_materia NVARCHAR(200))
    """,
    """
    CREATE TABLE dim_espacio (id_espacio INT PRIMARY KEY, edificio VARCHAR(50), aula VARCHAR(50), codigo_salon VARCHAR(100))
    """,
    """
    CREATE TABLE dim_tiempo (id_tiempo INT PRIMARY KEY, dia_codigo VARCHAR(10), dia_semana NVARCHAR(20), hora_inicio TIME, hora_fin TIME)
    """,
    """
    CREATE TABLE hechos_clase (
        id_hecho INT IDENTITY(1,1) PRIMARY KEY, 
        id_docente INT, id_materia INT, id_espacio INT, id_tiempo INT,
        nrc VARCHAR(20), clave VARCHAR(50), seccion VARCHAR(50), duracion_min INT
        -- Las FKs se añadirán al final
    )
    """
]

print(">> Creando tablas...")
for ddl in ddl_statements:
    conn.execute(text(ddl))
conn.commit()
print("✅ Tablas creadas correctamente.")


# ---------------------------------------------------------------------------
# 🧹 Limpieza y normalización (Se mantiene el código anterior)
# ---------------------------------------------------------------------------

# Renombrar columnas si existen
if "profesor" in dim_docente.columns:
    dim_docente.rename(columns={"profesor": "nombre_completo"}, inplace=True)
if "materia" in dim_materia.columns:
    dim_materia.rename(columns={"materia": "nombre_materia"}, inplace=True)

# Normalizar nombre de columna 'secc' -> 'seccion'
if "secc" in hechos_clase.columns:
    hechos_clase.rename(columns={"secc": "seccion"}, inplace=True)
elif "sección" in hechos_clase.columns:
    hechos_clase.rename(columns={"sección": "seccion"}, inplace=True)
elif "días" in hechos_clase.columns:
    hechos_clase.rename(columns={"días": "seccion"}, inplace=True)

# Reemplazar NaN y valores "nan" o "NaT" por None
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df = df.replace({np.nan: None, "nan": None, "NaN": None, "NaT": None})
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).replace(
                {"nan": None, "None": None, "NaN": None, "NaT": None, "": None}
            )
    return df

for df_name, df in {
    "dim_docente": dim_docente, "dim_materia": dim_materia,
    "dim_espacio": dim_espacio, "dim_tiempo": dim_tiempo,
    "hechos_clase": hechos_clase,
}.items():
    globals()[df_name] = clean_dataframe(df)
    print(f"NaN limpiados en {df_name}")

# ---------------------------------------------------------------------------
# 🧹 Limpieza previa en base de datos (TRUNCATE ahora funciona sin FKs)
# ---------------------------------------------------------------------------

print(">> Limpiando tablas con TRUNCATE...")
truncate_statements = [
    "TRUNCATE TABLE hechos_clase",
    "TRUNCATE TABLE dim_docente",
    "TRUNCATE TABLE dim_materia",
    "TRUNCATE TABLE dim_espacio",
    "TRUNCATE TABLE dim_tiempo"
]

try:
    with engine.begin() as t_conn:
        for stmt in truncate_statements:
            t_conn.execute(text(stmt))
        t_conn.commit()
    print("✅ Tablas limpiadas correctamente con TRUNCATE.")
except Exception as e:
    print(f"❌ ERROR al truncar tablas: {e}")


# ---------------------------------------------------------------------------
# 🧠 FUNCIÓN DE INSERCIÓN CORREGIDA (Usamos 'append' para todo después del TRUNCATE)
# ---------------------------------------------------------------------------

def insert_dataframe_to_sql_server(df: pd.DataFrame, table_name: str, conn: Connection):
    if df.empty:
        print(f"(⚠️ {table_name} está vacío, no se inserta nada)")
        return 0 
    
    rows_inserted = df.to_sql(
        name=table_name,
        con=engine, # Usamos 'engine' para todas las operaciones
        if_exists='append', # Usamos 'append' después del TRUNCATE
        index=False,
        chunksize=1000 
    )
    # Devolvemos el número real de filas del DataFrame para el mensaje de éxito
    return len(df)


# ---------------------------------------------------------------------------
# 🚀 CARGA DE DATOS
# ---------------------------------------------------------------------------

rows_inserted = insert_dataframe_to_sql_server(dim_docente, "dim_docente", engine)
print(f"✅ {rows_inserted} filas insertadas en dim_docente")

rows_inserted = insert_dataframe_to_sql_server(dim_materia, "dim_materia", engine)
print(f"✅ {rows_inserted} filas insertadas en dim_materia")

rows_inserted = insert_dataframe_to_sql_server(dim_espacio, "dim_espacio", engine)
print(f"✅ {rows_inserted} filas insertadas en dim_espacio")

rows_inserted = insert_dataframe_to_sql_server(dim_tiempo, "dim_tiempo", engine)
print(f"✅ {rows_inserted} filas insertadas en dim_tiempo")


rows_inserted = insert_dataframe_to_sql_server(hechos_clase, "hechos_clase", engine)
print(f"✅ {rows_inserted} filas insertadas en hechos_clase")


# ---------------------------------------------------------------------------
# 🔗 CREACIÓN FINAL DE CLAVES FORÁNEAS
# ---------------------------------------------------------------------------

print(">> Creando Claves Foráneas...")
fk_statements = [
    "ALTER TABLE hechos_clase ADD CONSTRAINT FK_Docente FOREIGN KEY (id_docente) REFERENCES dim_docente(id_docente);",
    "ALTER TABLE hechos_clase ADD CONSTRAINT FK_Materia FOREIGN KEY (id_materia) REFERENCES dim_materia(id_materia);",
    "ALTER TABLE hechos_clase ADD CONSTRAINT FK_Espacio FOREIGN KEY (id_espacio) REFERENCES dim_espacio(id_espacio);",
    "ALTER TABLE hechos_clase ADD CONSTRAINT FK_Tiempo FOREIGN KEY (id_tiempo) REFERENCES dim_tiempo(id_tiempo);"
]

try:
    with engine.begin() as t_conn:
        for stmt in fk_statements:
            t_conn.execute(text(stmt))
        t_conn.commit()
    print("✅ Claves Foráneas creadas exitosamente.")
except Exception as e:
    # Esto atrapará errores si las FKs ya existen de una ejecución previa
    print(f"⚠️ Las FKs ya existen o hubo un error al crearlas: {e}")


print("✅ Todos los datos cargados exitosamente en SQL Server.")

# ---------------------------------------------------------------------------
# 🔚 Cierre
# ---------------------------------------------------------------------------
conn.close()
print("🔚 Conexión a SQL Server cerrada.")

✅ Conexión a SQL Server establecida correctamente (Windows Auth).
>> Eliminando tablas existentes...
✅ Tablas eliminadas correctamente.
>> Creando tablas...
✅ Tablas creadas correctamente.
NaN limpiados en dim_docente
NaN limpiados en dim_materia
NaN limpiados en dim_espacio
NaN limpiados en dim_tiempo
NaN limpiados en hechos_clase
>> Limpiando tablas con TRUNCATE...
✅ Tablas limpiadas correctamente con TRUNCATE.
✅ 105 filas insertadas en dim_docente
✅ 96 filas insertadas en dim_materia
✅ 42 filas insertadas en dim_espacio
✅ 47 filas insertadas en dim_tiempo
✅ 1184 filas insertadas en hechos_clase
>> Creando Claves Foráneas...
✅ Claves Foráneas creadas exitosamente.
✅ Todos los datos cargados exitosamente en SQL Server.
🔚 Conexión a SQL Server cerrada.
